In [3]:
import time
from neo4j import GraphDatabase

In [2]:
uri = "neo4j+s://b073ebf5.databases.neo4j.io"
user = "neo4j"
password = "dFUBgoUZAXirgfzSyJT4z_dpznMWBU-g5Z53SHGtonE"

driver = GraphDatabase.driver(uri, auth=(user, password))

# Test de conexión
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Conexión exitosa con AuraDB' AS mensaje")
        for record in result:
            print(record["mensaje"])

test_connection()

Conexión exitosa con AuraDB


In [4]:
# Configura tu conexión a la base de datos de Neo4j Aura
URI = "neo4j+s://b073ebf5.databases.neo4j.io"
AUTH = ("neo4j", "dFUBgoUZAXirgfzSyJT4z_dpznMWBU-g5Z53SHGtonE")

# Diccionario de radios por categoría (en metros)
radios = {
    1: 600,
    2: 1200,
    3: 2400
}

# Diccionario para asignar el tipo de relación a cada categoría
relacion_por_categoria = {
    1: "CERCA_DE_CAT1",
    2: "CERCA_DE_CAT2",
    3: "CERCA_DE_CAT3"
}

# Inicializa el driver
driver = GraphDatabase.driver(URI, auth=AUTH)
driver.verify_connectivity()
print("Conexión a Neo4j establecida.")

Conexión a Neo4j establecida.


In [5]:
# Iterar sobre cada categoría y crear las relaciones
with driver.session() as session:
    for cat, radio in radios.items():
        relacion = relacion_por_categoria[cat]
        print(f"\n--- Creando relaciones de tipo :{relacion} con radio de {radio}m ---")
        
        # La query utiliza apoc.periodic.iterate para procesar los departamentos en lotes
        # para manejar grandes volúmenes de datos de forma segura.
        query = f"""
        CALL apoc.periodic.iterate(
            'MATCH (d:Departamento) RETURN d',
            'MATCH (p:POI)
             WHERE p.cat = $categoria AND point.distance(d.location, p.location) < $radio
             WITH d, p, point.distance(d.location, p.location) AS distancia
             CREATE (d)-[:{relacion} {{distancia_metros: distancia}}]->(p)',
            {{batchSize: 1000, parallel: false, params: {{categoria: $categoria, radio: $radio}}}}
        ) YIELD batches, total
        RETURN batches, total
        """
        
        start_time = time.time()
        result = session.run(query, categoria=cat, radio=radio).data()
        end_time = time.time()
        
        # Imprime los resultados del proceso
        batches = result[0]['batches']
        total = result[0]['total']
        duration = round(end_time - start_time, 2)
        
        print(f"Proceso finalizado para la categoría {cat}.")
        print(f"Total de departamentos procesados: {total}.")
        print(f"Tiempo total: {duration} segundos.")

# Cierra la conexión
driver.close()


--- Creando relaciones de tipo :CERCA_DE_CAT1 con radio de 600m ---
Proceso finalizado para la categoría 1.
Total de departamentos procesados: 25215.
Tiempo total: 783.51 segundos.

--- Creando relaciones de tipo :CERCA_DE_CAT2 con radio de 1200m ---
Proceso finalizado para la categoría 2.
Total de departamentos procesados: 25215.
Tiempo total: 379.84 segundos.

--- Creando relaciones de tipo :CERCA_DE_CAT3 con radio de 2400m ---


[#D894]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-b073ebf5-b541.production-orch-0067.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687))): OSError('No data')


SessionExpired: Failed to read from defunct connection IPv4Address(('si-b073ebf5-b541.production-orch-0067.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))